# Problèmes

### 1)
Primal: $\underset{x \in \mathbb{R}^3}{\min} \frac{1}{2} x' Q x + c' x ~~~~~~~~ s.t. ~ A x = b ~, ~~ x \ge 0$

with  $Q = \begin{bmatrix} 6 & 2 & 1\\ 2 & 5 & 2\\ 1 & 2 & 4 \end{bmatrix} $, $c = \begin{bmatrix} -8 \\ -3 \\ -3 \end{bmatrix}$, $A = \begin{bmatrix} 1 & 0 &1 \\ 0 & 1 & 1 \end{bmatrix}$ , and $b = \begin{bmatrix} 3 \\ 0 \end{bmatrix} $.


### 2) 
AFIRO        28 rows    32 cols        88 nonzeros       794 bytes      -4.6475314286E+02 optimalvalue

In [1]:
using LinearAlgebra
using LaTeXStrings
using DataFrames
using DataStructures
using QuadraticModels
using Printf
using SparseArrays
using BenchmarkTools
using NLPModels
using LinearOperators
using QPSReader
using SolverTools
using SolverBenchmark

# Mehrotra formulation plus générale:

min $c_0 + c'x + \frac{1}{2} x'Hx ~~~~$ s.t. $ Ax = b $ and $l \le x \le u$ 

In [181]:
function display_results(result)
    # fonction pour l'affichage
    println("\n-----------------------------------------------------------------------")
    println("------------------------------- RESULTS -------------------------------")
    result
end

function init_x0(lvar, uvar)
    # choice of an init point x0
    x0 = zeros(length(lvar))
    for i=1:length(x0)
        if lvar[i] == -Inf && uvar[i] == Inf
            x0[i] = 0.
        elseif lvar[i] == -Inf && uvar[i] != Inf
            x0[i] = uvar[i] - 1.
        elseif lvar[i] != -Inf && uvar[i] == Inf
            x0[i] = lvar[i] + 1.
        else
            x0[i] = (lvar[i] + uvar[i]) / 2 
        end
    end  
    return x0
end

function init_x0_lsq(A, b, lvar, uvar)
    x_tilde = A\b
    n = length(x_tilde)
    for i=1:n
        if x_tilde[i] <= lvar[i]
            x_tilde[i] = lvar[i] + 1.
        elseif uvar[i] <= x_tilde[i]
            x_tilde[i] = uvar[i] - 1.
        end
        if !(lvar[i] < x_tilde[i] < uvar[i])
            x_tilde[i] = (lvar[i] + uvar[i]) / 2.
        end
    end

    return x_tilde
end

function starting_points(Q, A, b, c, lvar, uvar, ilow, iupp, irng, J_augm, ρ, n_rows, n_cols)

    tmp_diag = vcat(-ρ*ones(n_cols),spzeros(n_rows))

    J_fact = ldlt(Symmetric(J_augm-Diagonal(tmp_diag), :L))
    init_xλ = J_fact \ [c ; b ]
    x0 = init_xλ[1:n_cols]
    λ0 = init_xλ[n_cols+1:end]
    s0_l, s0_u = zeros(n_cols), zeros(n_cols)
    dual_val = Q*x0 - A'*λ0 + c
    s0_l[ilow] = dual_val[ilow]
    s0_u[iupp] = -dual_val[iupp]

    x0_l1, x0_u1 = x0[ilow], x0[iupp]
    if length(ilow) == 0
        δx_l1, δs_l1 = 0., 0.
    else
        δx_l1 = max(-1.5*minimum(x0_l1 - lvar[ilow]), 0.)
        δs_l1 = max(-1.5*minimum(s0_l[ilow]), 0.)
    end

    if length(iupp) == 0
        δx_u1, δs_u1 = 0., 0.
    else
        δx_u1 = max(-1.5*minimum((uvar[iupp] - x0_u1)), 0.)
        δs_u1 = max(-1.5*minimum(s0_u[iupp]), 0.)
    end

    x0_l1 .+= δx_l1
    x0_u1 .-= δx_u1
    s0_l1 = s0_l[ilow] .+ δs_l1
    s0_u1 = s0_u[iupp] .+ δs_u1
    xs_l1, xs_u1 = s0_l1'*(x0_l1 - lvar[ilow]), s0_u1'*(uvar[iupp] - x0_u1)
    if length(ilow) == 0
        δx_l2, δs_l2 = 0., 0.
    else
        δx_l2 = δx_l1 + 0.5 *  xs_l1 / sum(s0_l1)
        δs_l2 = δs_l1 + 0.5 * xs_l1 / sum(x0_l1-lvar[ilow])
    end
    if length(iupp) == 0
        δx_u2, δs_u2 = 0., 0.
    else
        δx_u2 = δx_u1 + 0.5 *  xs_u1 / sum(s0_u1)
        δs_u2 = δs_u1 + 0.5 * xs_u1 / sum(uvar[iupp]-x0_u1)
    end

    x0[ilow] .+= δx_l2 
    x0[iupp] .-= δx_u2
    s0_l[ilow] = s0_l[ilow] .+ max(δs_l2, δs_u2)
    s0_u[iupp] = s0_u[iupp] .+ max(δs_l2, δs_u2)

    for i in irng
        if (lvar[i] < x0[i] < uvar[i]) == false
            x0[i] = (lvar[i] + uvar[i]) / 2.
        end
    end

    @assert all(x0 .> lvar) && all(x0 .< uvar)
    @assert all(s0_l[ilow] .> 0) && all(s0_u[iupp] .> 0)

    return x0, λ0, s0_l, s0_u
end



function starting_points2(Q, A, b, c, lvar, uvar, ilow, iupp, irng, J_augm, ρ, n_rows, n_cols)

    tmp_diag = vcat(-ρ*ones(n_cols),spzeros(n_rows))

    J_fact = ldlt(Symmetric(J_augm-Diagonal(tmp_diag), :L))
    init_xλ = J_fact \ [c ; b ]
    x0 = init_xλ[1:n_cols]
    λ0 = init_xλ[n_cols+1:end]
    s0_l, s0_u = zeros(n_cols), zeros(n_cols)
    dual_val = Q*x0 - A'*λ0 + c
    s0_l[ilow] = dual_val[ilow]
    s0_u[iupp] = -dual_val[iupp]

    x0_l1, x0_u1 = x0[ilow], x0[iupp]

    for i in ilow
        δx_l = max(-1.5*(x0[i] - lvar[i]), 0.)
        δs_l = max(-1.5*s0_l[i], 0.)
        x0[i] += δx_l 
        s0_l[i] += δs_l 
    end

    for i in iupp
        δx_u = max(-1.5*(uvar[i] - x0[i]), 0.)
        δs_u = max(-1.5*s0_u[i], 0.)
        x0[i] -= δx_u 
        s0_u[i] += δs_u 
    end

    for i in irng
        if (lvar[i] < x0[i] < uvar[i]) == false
            x0[i] = (lvar[i] + uvar[i]) / 2.
        end
    end

    @assert all(x0 .> lvar) && all(x0 .< uvar)
    @assert all(s0_l[ilow] .> 0) && all(s0_u[iupp] .> 0)

    return x0, λ0, s0_l, s0_u
end



starting_points2 (generic function with 1 method)

In [182]:
function compute_α_dual(v, dir_v)
    n = length(v)
    if n == 0
        return 1.
    end
    α = 1.
    for i=1:n
        if dir_v[i] < 0.
            α_new = -v[i] * 0.999 / dir_v[i]
            if α_new < α
                α = α_new
            end
        end
    end
    return α
end


    
function compute_α_primal(v, dir_v, lvar, uvar)
    n = length(v)
    α_l, α_u = 1., 1.
    for i=1:n
        if dir_v[i] > 0.
            α_u_new = (uvar[i] - v[i]) * 0.999 / dir_v[i]
            if α_u_new < α_u
                α_u = α_u_new
            end
        elseif dir_v[i] < 0.
            α_l_new = (lvar[i] - v[i]) * 0.999 / dir_v[i]
            if α_l_new < α_l
                α_l = α_l_new
            end
        end
    end
    return min(α_l, α_u)
end

function compute_μ(x_l, x_u, s_l, s_u, lvar, uvar, nb_low, nb_upp)
    #x_l coordinates of x corresponding to finite lower bounds ( resp. finite upper bounds for x_u)
    # arguments must have finite bounds 
    return (s_l' * (x_l-lvar) + s_u' * (uvar-x_u)) / (nb_low + nb_upp)
end


function is_in_Neighborhood_inf(gamma, x_l, x_u, s_l, s_u, lvar, uvar)
    # check if the current point is in N_inf(gamma)
    # true : (xi_l - lvari) * si_l >= gamma mu   and   (uvari - xi_u) * si_u >= gamma mu 
    mu = Compute_mu(x_l, x_u, s_l, s_u, lvar, uvar)
    for i=1:length(x_l)
        if (x_l[i] - lvar[i]) * s_l[i] < gamma*mu
            return false
        end
    end
    for i=1:length(x_u)
        if (uvar[i] - x_u[i]) * s_u[i] < gamma*mu
            return false
        end
    end
    return true
end

is_in_Neighborhood_inf (generic function with 1 method)

In [183]:
function solve_augmented_system_aff(J_fact, rc, rb, x, s_l, s_u, ilow, 
                                    iupp, lvar, uvar, n_cols)
    
    F_x_λ_aff = [-rc 
                 -rb]
    F_x_λ_aff[ilow] += s_l[ilow]
    F_x_λ_aff[iupp] -= s_u[iupp]
    
    Δ_x_λ_aff = J_fact\F_x_λ_aff
    Δ_aff = [Δ_x_λ_aff
             -s_l[ilow] - s_l[ilow].*Δ_x_λ_aff[1:n_cols][ilow]./(x[ilow]-lvar[ilow])
             - s_u[iupp] - s_u[iupp].*Δ_x_λ_aff[1:n_cols][iupp]./(x[iupp]-uvar[iupp])]
    return Δ_aff
end

function solve_augmented_system_cc(J_fact, σ, μ,x, s_l, s_u, ilow, iupp,  lvar, uvar,
                                   Δ_aff, n_cols, n_rows, n_low)


    rxs_l = -σ*μ .+ Δ_aff[1:n_cols][ilow].*Δ_aff[n_rows+n_cols+1: n_rows+n_cols+n_low]
    rxs_u = σ*μ .+ Δ_aff[1:n_cols][iupp].*Δ_aff[n_rows+n_cols+n_low+1: end]

    F_x_λ_cc = zeros(n_cols+n_rows,1)
    F_x_λ_cc[ilow] += rxs_l./(x[ilow]-lvar[ilow])
    F_x_λ_cc[iupp] -= rxs_u./(x[iupp]-uvar[iupp])

#     F_x_λ_cc = [ I_low*inv_X_L*rxs_l + I_op_upp*inv_X_U*rxs_u
#                                    Or                 ]
    Δ_x_λ_cc = J_fact\F_x_λ_cc
    Δ_cc = [Δ_x_λ_cc
            -(rxs_l+s_l[ilow].*Δ_x_λ_cc[1:n_cols][ilow])./(x[ilow]-lvar[ilow])
            -(rxs_u+s_u[iupp].*Δ_x_λ_cc[1:n_cols][iupp])./(x[iupp]-uvar[iupp]) ]
    return Δ_cc
end


solve_augmented_system_cc (generic function with 1 method)

In [184]:
function mehrotraPCQuadBounds(QM; max_iter=300, ϵ_pdd=1e-8, ϵ_rb=1e-6, ϵ_rc=1e-6,
                              tol_Δx=1e-16, ϵ_μ=0., max_time=60., display=true)
    
    start_time = time()
    elapsed_time = 0.0
    
    # get variables from QuadraticModel
    lvar, uvar = QM.meta.lvar, QM.meta.uvar
    n_cols = length(lvar)
    Oc = zeros(n_cols)
    ilow, iupp = [QM.meta.ilow; QM.meta.irng], [QM.meta.iupp; QM.meta.irng] # finite bounds index
    n_low, n_upp = length(ilow), length(iupp) # number of finite constraints
    A = jac(QM, Oc)
    Arows, Acols, Avals = findnz(A)
    n_rows, n_cols = size(A) 
    Q = hess(QM, Oc)  # lower triangular
    Qrows, Qcols, Qvals = findnz(Q)
    c = grad(QM, Oc)
    c0 = obj(QM, Oc)
    @assert QM.meta.lcon == QM.meta.ucon # equality constraint (Ax=b)
    b = QM.meta.lcon
    
    # init regularization values
    ρ, δ = 1.e-8, 1.e-6

    J_augmrows = vcat(Qrows, Arows.+n_cols, n_cols+1:n_cols+n_rows, 1:n_cols)
    J_augmcols = vcat(Qcols, Acols, n_cols+1:n_cols+n_rows, 1:n_cols)
    tmp_diag = zeros(n_cols)
    J_augmvals = vcat(-Qvals, Avals, δ*ones(n_rows), tmp_diag)
    J_augm = sparse(J_augmrows, J_augmcols, J_augmvals)
    
    x, λ, s_l, s_u = starting_points(Q, A, b, c, lvar, uvar, ilow, iupp, 
                                     QM.meta.irng, J_augm, 1e-6, n_rows, n_cols)
    
#     x = init_x0_lsq(A, b, lvar, uvar)
#     @assert all(x .> lvar) && all(x .< uvar)
#     s_l, s_u = copy(Oc), copy(Oc)
#     s_l[ilow] .= 1.
#     s_u[iupp] .= 1.
    Qx = Q * x
#     λ = sparse(A') \ (c+Qx) # least square initialisation, s_0 = stilde_0
    
    rb = A * x - b
    rc = -Qx + A' * λ + s_l - s_u - c
    μ = compute_μ(x[ilow], x[iupp], 
                  s_l[ilow], s_u[iupp], 
                  lvar[ilow], uvar[iupp],
                  n_low, n_upp)

    k = 0

    # stopping criterion
    xTQx_2 = x' * Qx / 2.
    cTx = c' * x
    pri_obj = xTQx_2 + cTx + c0
    dual_obj = b' * λ - xTQx_2 + s_l[ilow]'*lvar[ilow] - s_u[iupp]'*uvar[iupp] +c0
    pdd = abs(pri_obj - dual_obj ) / (1. + abs(pri_obj)) 
    max_rc, max_rb = norm(rc, Inf), norm(rb, Inf)
    optimal = pdd < ϵ_pdd && max_rc < ϵ_rc && max_rb < ϵ_rb
    n_Δx = 0.
    small_Δx, small_μ = false, μ < ϵ_μ
    Δt = time() - start_time
    tired = Δt > max_time
    

    # display
    if display == true
        @info log_header([:k, :pri_obj, :pdd, :max_rb, :max_rc, :n_Δx, :μ],
                         [Int, Float64, Float64, Float64, Float64, Float64, Float64, Float64],
                        hdr_override=Dict(:k=>"Iter", :pri_obj=>"primal", :pdd=>"pdd",
                                          :max_rb=>"rb cond", :max_rc=>"rc cond",
                                          :n_Δx=>"‖Δx‖", :μ=>"μ"))
        @info log_row([k, pri_obj, pdd, max_rb, max_rc, n_Δx, μ])
    end

    while k<max_iter && !optimal && !small_Δx && !small_μ && !tired
        
            
            # Affine scaling direction
        tmp_diag .= -ρ
        tmp_diag[ilow] -= s_l[ilow] ./ (x[ilow] - lvar[ilow]) 
        tmp_diag[iupp] -= s_u[iupp] ./ (uvar[iupp] - x[iupp]) 
        J_augmvals[end-n_cols+1:end] = tmp_diag

        J_augm = sparse(J_augmrows, J_augmcols, J_augmvals)
    
        J_fact = ldlt(Symmetric(J_augm, :L))
        #J_fact = lu(J_augm, check=true)
        
        Δ_aff = solve_augmented_system_aff(J_fact, rc, rb, x, s_l, s_u, ilow, 
                                           iupp, lvar, uvar, n_cols)

        α_aff_pri = compute_α_primal(x, Δ_aff[1:n_cols], lvar, uvar)
        α_aff_dual_l = compute_α_dual(s_l[ilow], Δ_aff[n_rows+n_cols+1: n_rows+n_cols+n_low])
        α_aff_dual_u = compute_α_dual(s_u[iupp], Δ_aff[n_rows+n_cols+n_low+1:end])

        # alpha_aff_dual_final is the min of the 2 alpha_aff_dual
        α_aff_dual_final = min(α_aff_dual_l, α_aff_dual_u)
        
        μ_aff = compute_μ(x[ilow] + α_aff_pri * Δ_aff[1:n_cols][ilow],
                          x[iupp] + α_aff_pri * Δ_aff[1:n_cols][iupp],
                          s_l[ilow] + α_aff_dual_final * Δ_aff[n_rows+n_cols+1: n_rows+n_cols+n_low],
                          s_u[iupp] + α_aff_dual_final * Δ_aff[n_rows+n_cols+n_low+1: end],
                          lvar[ilow], uvar[iupp],
                          n_low, n_upp)
        
        σ = (μ_aff / μ)^3

        # corrector and centering step
        
        Δ_cc = solve_augmented_system_cc(J_fact, σ, μ,x, s_l, s_u, ilow, iupp,  lvar, uvar,
                                         Δ_aff, n_cols, n_rows, n_low)


        Δ = Δ_aff + Δ_cc # final direction
        
        α_pri = compute_α_primal(x, Δ[1:n_cols], lvar, uvar)
        α_dual_l = compute_α_dual(s_l[ilow], Δ[n_rows+n_cols+1: n_rows+n_cols+n_low])
        α_dual_u = compute_α_dual(s_u[iupp], Δ[n_rows+n_cols+n_low+1: end])

        α_dual_final = min(α_dual_l, α_dual_u)

        # new parameters
        x += α_pri * Δ[1:n_cols]
        λ += α_dual_final * Δ[n_cols+1: n_rows+n_cols]
        s_l[ilow] += α_dual_final * Δ[n_rows+n_cols+1: n_rows+n_cols+n_low]
        s_u[iupp] += α_dual_final * Δ[n_rows+n_cols+n_low+1: end]
        
        n_Δx = α_pri * norm(Δ[1:n_cols])
        μ = compute_μ(x[ilow], x[iupp], 
                      s_l[ilow], s_u[iupp], 
                      lvar[ilow], uvar[iupp],
                      n_low, n_upp)
        Qx = Q * x
        xTQx_2 = x' * Qx / 2.
        cTx = c' * x
        pri_obj = xTQx_2 + cTx + c0
        dual_obj = b' * λ - xTQx_2 + s_l[ilow]'*lvar[ilow] - s_u[iupp]'*uvar[iupp] +c0

        rb = A * x - b
        rc = -Qx + A' * λ + s_l - s_u - c
        
        # update stopping criterion values:
        
        pdd = abs(pri_obj - dual_obj ) / (1. + abs(pri_obj)) 
        max_rc, max_rb = norm(rc, Inf), norm(rb, Inf)
        optimal = pdd < ϵ_pdd && max_rb < ϵ_rb && max_rc < ϵ_rc
        small_Δx, small_μ = n_Δx < tol_Δx, μ < ϵ_μ
        k += 1
        
                    ### case ρ=δ and ρ /= 10 each iter
#         if ρ >= 1e-7
#             for j in n_cols+1:n_rows+n_cols
#                 J_augm[j,j] /= 10.
#             end
#             ρ /= 10.
#         end
        Δt = time() - start_time
        tired = Δt > max_time
        
        if display == true
            @info log_row([k, pri_obj, pdd, max_rb, max_rc, n_Δx, μ])
        end
        
        
    end
    
    elapsed_time = time() - start_time
    
    if k>= max_iter
        status = :max_iter
    elseif tired
        status = :max_time 
    else
        status = :acceptable
    end
    
    stats = GenericExecutionStats(status, QM, solution = x,
                                  objective = pri_obj , 
                                  dual_feas = max_rc, 
                                  primal_feas = max_rb,
                                  multipliers = λ,
                                  multipliers_L = s_l,
                                  multipliers_U = s_u,
                                  iter = k, elapsed_time=elapsed_time)
    return stats
end



mehrotraPCQuadBounds (generic function with 1 method)

### bibliothèque QuadraticModels

optimize $c_0 + c'x + \frac{1}{2} x'Hx ~~~~$ s.t. $~~L \le Ax \le U$ and $l \le x \le u$ 

Ici L = lcon, U = ucon, l = lvar, u = uvar

In [185]:
# probleme1
Q = [6. 2 1
     2 5 2
     1 2 4]
c = [-8.; -3; -3]
c0 = 0.
A = [1. 0 1
    0 1 1]
b = [0.; 3]
lvar = [0.;0;0]
uvar = [Inf; Inf; Inf]
lcon = b
ucon = b

x01 = [1.; 2.; 3.];


In [186]:
QM = QuadraticModel(c, Q, A=A, lcon=lcon, ucon=ucon, lvar=lvar, uvar=uvar, x0=x01, c0=c0, name="QM1")
SM = SlackModel(QM)
typeof(SM)

QuadraticModel

In [187]:
stats_mpc1 =  mehrotraPCQuadBounds(SM)
println(stats_mpc1)

┌ Info:   Iter    primal       pdd   rb cond   rc cond      ‖Δx‖         μ  
└ @ Main In[184]:68


Generic Execution stats
  status: solved to within acceptable tolerances
  objective value: 13.499999999923611
  primal feasibility: 5.142553050063725e-12
  dual feasibility: 6.486700065977402e-9
  solution: [3.233003491705889e-13  2.999999999993769  1.0883346895065592e-12]
  multipliers: [-13.350622652649395  11.99999999348161]
  multipliers_L: [5.350622652138543  1.1802956709797145e-12  4.350622659155794]
  multipliers_U: [0.0  0.0  0.0]
  iterations: 5
  elapsed time: 2.4749999046325684


┌ Info:  0.0e+00   4.7e+01   2.0e+00   3.6e+00   1.1e+00   0.0e+00   2.7e+01
└ @ Main In[184]:73
┌ Info:  1.0e+00   1.4e+01   1.2e+00   1.1e-01   5.0e+00   2.8e+00   1.3e+00
└ @ Main In[184]:162
┌ Info:  2.0e+00   1.4e+01   2.7e-03   1.2e-03   2.3e-01   1.6e-01   2.9e-03
└ @ Main In[184]:162
┌ Info:  3.0e+00   1.4e+01   4.5e-04   1.4e-06   2.2e-03   1.1e-03   3.3e-06
└ @ Main In[184]:162
┌ Info:  4.0e+00   1.3e+01   8.9e-07   1.4e-09   4.3e-06   1.2e-06   3.3e-09
└ @ Main In[184]:162
┌ Info:  5.0e+00   1.3e+01   1.3e-09   5.1e-12   6.5e-09   2.2e-09   3.3e-12
└ @ Main In[184]:162


In [188]:
#pb verif

QM_verif = QuadraticModel(c*(-1), Q, A=A*(-1), lcon=lcon, ucon=ucon, lvar=-uvar, uvar=lvar, x0=x01, c0=c0, name="QM1")
SM_verif = SlackModel(QM_verif)
stats_mpc_verif =  mehrotraPCQuadBounds(SM_verif);
println(stats_mpc_verif)

Generic Execution stats
  status: solved to within acceptable tolerances
  objective value: 13.499999999923611
  primal feasibility: 5.142553050063725e-12
  dual feasibility: 6.486700065977402e-9
  solution: [-3.233003491705889e-13  -2.999999999993769  -1.0883346895065592e-12]
  multipliers: [-13.350622652649395  11.99999999348161]
  multipliers_L: [0.0  0.0

┌ Info:   Iter    primal       pdd   rb cond   rc cond      ‖Δx‖         μ  
└ @ Main In[184]:68
┌ Info:  0.0e+00   4.7e+01   2.0e+00   3.6e+00   1.1e+00   0.0e+00   2.7e+01
└ @ Main In[184]:73
┌ Info:  1.0e+00   1.4e+01   1.2e+00   1.1e-01   5.0e+00   2.8e+00   1.3e+00
└ @ Main In[184]:162
┌ Info:  2.0e+00   1.4e+01   2.7e-03   1.2e-03   2.3e-01   1.6e-01   2.9e-03
└ @ Main In[184]:162
┌ Info:  3.0e+00   1.4e+01   4.5e-04   1.4e-06   2.2e-03   1.1e-03   3.3e-06
└ @ Main In[184]:162
┌ Info:  4.0e+00   1.3e+01   8.9e-07   1.4e-09   4.3e-06   1.2e-06   3.3e-09
└ @ Main In[184]:162
┌ Info:  5.0e+00   1.3e+01   1.3e-09   5.1e-12   6.5e-09   2.2e-09   3.3e-12
└ @ Main In[184]:162


  0.0]
  multipliers_U: [5.350622652138543  1.1802956709797145e-12  4.350622659155794]
  iterations: 5
  elapsed time: 0.0


In [189]:
### pb verif 2
c2 = [1.; 0; 0]
A2 = Matrix([1., 1, 1]')
b2 = [1.];
QM_verif2 = QuadraticModel(c2, zeros(3,3), A=A2, lcon=b2, ucon=b2, lvar=[0.;0;0], 
                uvar=[Inf; Inf; Inf], x0=x01, c0=c0, name="QMverfi2")
SM_verif2 = SlackModel(QM_verif2)
stats_mpc_verif2 =  mehrotraPCQuadBounds(SM_verif2);
println(stats_mpc_verif2)

Generic Execution stats
  status: solved to within acceptable tolerances
  objective value: 5.352156597626284e-10
  primal feasibility: 5.961897642237091e-13
  dual feasibility: 1.2878587085651816e-14
  solution: [5.352156597626284e-10  0.4999999997320941  0.4999999997320941]
  multipliers: [-1.9882315318729204e-10]
  multipliers_L: [1.0000000001988103  1.9882969377120522e-10  1.9882969377120522e-10]
  multipliers_U: [0.0  0.0  0.0]
  iterations: 6
  elapsed time: 0.002000093460083008


┌ Info:   Iter    primal       pdd   rb cond   rc cond      ‖Δx‖         μ  
└ @ Main In[184]:68
┌ Info:  0.0e+00   1.6e+06   1.0e+00   2.9e+06   9.7e-01   0.0e+00   1.2e+06
└ @ Main In[184]:73
┌ Info:  1.0e+00   1.8e+03   1.0e+00   3.1e+03   1.6e-02   1.9e+06   1.3e+03
└ @ Main In[184]:162
┌ Info:  2.0e+00   1.8e+00   8.7e-01   2.5e+00   1.8e-05   2.0e+03   1.3e+00
└ @ Main In[184]:162
┌ Info:  3.0e+00   2.4e-01   3.5e-01   4.4e-07   1.6e-08   1.7e+00   1.5e-01
└ @ Main In[184]:162
┌ Info:  4.0e+00   5.4e-04   7.3e-04   2.0e-07   2.4e-09   2.9e-01   2.4e-04
└ @ Main In[184]:162
┌ Info:  5.0e+00   5.4e-07   7.3e-07   4.0e-10   7.7e-12   6.5e-04   2.4e-07
└ @ Main In[184]:162
┌ Info:  6.0e+00   5.4e-10   7.3e-10   6.0e-13   1.3e-14   6.6e-07   2.4e-10
└ @ Main In[184]:162


# Lecture des donnees .SIF

In [190]:

function createQuadraticModel(qpdata; name="qp_pb")
    return QuadraticModel(qpdata.c, qpdata.qrows, qpdata.qcols, qpdata.qvals,
            Arows=qpdata.arows, Acols=qpdata.acols, Avals=qpdata.avals, 
            lcon=qpdata.lcon, ucon=qpdata.ucon, lvar=qpdata.lvar, uvar=qpdata.uvar,
            c0=qpdata.c0, name=name)
end

function displayQuadraticModel(QM)
    #println("A = ", Matrix(jac(QM, QM.meta.x0)))
    #println("Q = ", Matrix(hess(QM, QM.meta.x0)))
    println("lvar = ", QM.meta.lvar)
    println("uvar = ", QM.meta.uvar)
    println("x0 = ", QM.meta.x0)
    #println("lcon = ", QM.meta.lcon)
    #println("ucon = ", QM.meta.ucon)
end

displayQuadraticModel (generic function with 1 method)

# Benchmark

In [191]:
path_pb = "C:\\Users\\Geoffroy Leconte\\Documents\\cours\\TFE\\code\\problemes_netlib"
afiro = string(path_pb, "\\AFIRO.SIF")

"C:\\Users\\Geoffroy Leconte\\Documents\\cours\\TFE\\code\\problemes_netlib\\AFIRO.SIF"

In [192]:
qpdata2 = readqps(afiro)  # obj   -4.6475314286E+02
qpdata2.contypes
QM2 = createQuadraticModel(qpdata2)
SM2 = SlackModel(QM2);
SM2.meta

┌ Info: Using 'AFIRO' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'COST' as objective (l. 47)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'B' as RHS (l. 96)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470


  Problem name: qp_pb-slack
   All variables: ████████████████████ 51     All constraints: ████████████████████ 27    
            free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                 free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           lower: █████████████⋅⋅⋅⋅⋅⋅⋅ 32               lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           upper: ████████⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 19               upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
         low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0              low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                fixed: ████████████████████ 27    
          infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0               infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            nnzh: (100.00% sparsity)   0               linear: ████████████████████ 27    
                                                    nonlinear: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
                                                         nnzj: ( 92.59% sparsity)   102   



In [193]:
stats_mpc2 =  mehrotraPCQuadBounds(SM2)
println(stats_mpc2)

Generic Execution stats
  status: solved to within acceptable tolerances
  objective value: -464.75314285333866
  primal feasibility: 1.7053025658242404e-13
  dual feasibility: 7.771561172376096e-14
  solution: [79.99999999973801  25.499999999443926  54.50000000029404  84.79999999972244 ⋯ 102.40030236680133]
  multipliers: [-0.628571428587964  5.041533948240811e-11  -0.344771428536912  -0.22857142857736482 ⋯ -4.865439454081846e-13]
  multipliers_L: [1.2918838399401167e-12  1.0612299500435577e-11  1.686251661411353e-12  1.2191748047374688e-12 ⋯ 0.0]
  multipliers_U: [0.0  0.0  0.0  0.0 ⋯ 4.780827710792579e-13]
  iterations: 11
  elapsed time: 0.006999969482421875


┌ Info:   Iter    primal       pdd   rb cond   rc cond      ‖Δx‖         μ  
└ @ Main In[184]:68
┌ Info:  0.0e+00   1.0e+08   1.0e+00   7.8e+07   4.2e+00   0.0e+00   1.9e+07
└ @ Main In[184]:73
┌ Info:  1.0e+00   1.1e+07   1.0e+00   1.5e+07   1.2e-01   2.5e+07   3.5e+06
└ @ Main In[184]:162
┌ Info:  2.0e+00   1.6e+04   1.8e+00   1.8e+04   2.9e-02   7.6e+06   4.6e+03
└ @ Main In[184]:162
┌ Info:  3.0e+00   1.6e+03   6.5e+00   2.0e+03   3.4e-05   8.3e+03   6.2e+02
└ @ Main In[184]:162
┌ Info:  4.0e+00  -1.2e+02   3.8e+01   2.6e+01   1.4e-05   1.1e+03   9.8e+01
└ @ Main In[184]:162
┌ Info:  5.0e+00  -2.0e+02   4.3e+00   5.6e-06   2.3e-06   6.7e+01   1.7e+01
└ @ Main In[184]:162
┌ Info:  6.0e+00  -4.1e+02   4.5e-01   1.5e-06   2.6e-06   4.2e+02   3.6e+00
└ @ Main In[184]:162
┌ Info:  7.0e+00  -4.6e+02   7.7e-02   6.1e-07   1.0e-06   9.2e+01   6.9e-01
└ @ Main In[184]:162
┌ Info:  8.0e+00  -4.6e+02   7.7e-03   2.1e-07   8.6e-08   1.8e+01   7.0e-02
└ @ Main In[184]:162
┌ Info:  9.0e+00  -4.6

In [57]:
@benchmark mehrotraPCQuadBounds(SM2,  display=false)

BenchmarkTools.Trial: 
  memory estimate:  1.38 MiB
  allocs estimate:  5008
  --------------
  minimum time:     1.121 ms (0.00% GC)
  median time:      2.737 ms (0.00% GC)
  mean time:        2.860 ms (2.78% GC)
  maximum time:     8.522 ms (33.08% GC)
  --------------
  samples:          1743
  evals/sample:     1

In [194]:
# problem 3   kb2    obj  -1.7499001299E+03
kb2 = string(path_pb, "\\KB2.SIF")
qpdata3 = readqps(kb2)
QM3 = createQuadraticModel(qpdata3)
SM3 = SlackModel(QM3)
SM3.meta

┌ Info: Using 'KB2' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'FAT7..J.' as objective (l. 20)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using '77BOUND' as BOUNDS (l. 227)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:622


  Problem name: qp_pb-slack
   All variables: ████████████████████ 68     All constraints: ████████████████████ 43    
            free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                 free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           lower: ██████████████⋅⋅⋅⋅⋅⋅ 47               lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           upper: ████⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 12               upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
         low/upp: ███⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 9              low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                fixed: ████████████████████ 43    
          infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0               infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            nnzh: (100.00% sparsity)   0               linear: ████████████████████ 43    
                                                    nonlinear: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
                                                         nnzj: ( 89.30% sparsity)   313   



In [195]:
stats_mpc3 =  mehrotraPCQuadBounds(SM3);
println(stats_mpc3)

Generic Execution stats
  status: solved to within acceptable tolerances
  objective value: -1749.9001296069464
  primal feasibility: 1.0520295745664043e-9
  dual feasibility: 2.0843426572507855e-10
  solution: [0.8118215777798173  2.0809898685692808e-7  2.8347831727631393e-7  7.257066643775176e-8 ⋯ -783.4591288785368]
  multipliers: [17.269208082301336  17.214101827050303  16.942045682829416  16.6598385398139 ⋯ -5.8076737223977134e-11]
  multipliers_L: [2.860560183032895e-9 

┌ Info:   Iter    primal       pdd   rb cond   rc cond      ‖Δx‖         μ  
└ @ Main In[184]:68
┌ Info:  0.0e+00   1.4e+06   1.0e+00   5.6e+09   1.4e+01   0.0e+00   1.0e+08
└ @ Main In[184]:73
┌ Info:  1.0e+00   1.4e+06   1.1e+03   5.6e+09   2.0e+00   2.2e+04   1.6e+08
└ @ Main In[184]:162
┌ Info:  2.0e+00   4.9e+04   1.3e+02   1.2e+08   1.5e-01   7.2e+07   3.1e+06
└ @ Main In[184]:162
┌ Info:  3.0e+00  -3.5e+02   2.1e+03   1.7e+06   1.8e-02   2.8e+06   5.2e+04
└ @ Main In[184]:162
┌ Info:  4.0e+00  -1.2e+03   2.4e+02   2.3e+05   2.1e-04   3.8e+04   9.6e+03
└ @ Main In[184]:162
┌ Info:  5.0e+00  -1.2e+03   1.1e+02   7.9e+04   1.0e-04   5.1e+03   3.8e+03
└ @ Main In[184]:162
┌ Info:  6.0e+00  -1.4e+03   1.6e+01   3.1e+03   2.1e-05   4.1e+03   3.5e+02
└ @ Main In[184]:162
┌ Info:  7.0e+00  -1.4e+03   7.4e+00   7.7e+02   1.0e-05   7.6e+02   1.5e+02
└ @ Main In[184]:162
┌ Info:  8.0e+00  -1.6e+03   8.3e-01   1.4e-03   6.3e-06   1.0e+03   1.7e+01
└ @ Main In[184]:162
┌ Info:  9.0e+00  -1.7

 0.06381185642665568  0.04896173060985567  0.10208332010059237 ⋯ 0.0]
  multipliers_U: [0.0  8.556023646619279e-10  0.0  0.0 ⋯ 1.0807589233980402e-11]
  iterations: 17
  elapsed time: 0.031999826431274414


In [344]:
@benchmark mehrotraPCQuadBounds(SM3, display = false)

BenchmarkTools.Trial: 
  memory estimate:  2.76 MiB
  allocs estimate:  6064
  --------------
  minimum time:     2.074 ms (0.00% GC)
  median time:      2.136 ms (0.00% GC)
  mean time:        2.296 ms (4.08% GC)
  maximum time:     5.364 ms (35.17% GC)
  --------------
  samples:          2175
  evals/sample:     1

In [196]:
# problem 4 SC50A  obj  -6.4575077059E+01
SC50A = string(path_pb, "\\25FV47.SIF")
qpdata4 = readqps(SC50A)
QM4 = createQuadraticModel(qpdata4)
SM4 = SlackModel(QM4)
SM4.meta

┌ Info: Using '25FV47' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'R0000' as objective (l. 21)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using '.00001' as RHS (l. 6793)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470


  Problem name: qp_pb-slack
   All variables: ████████████████████ 1876   All constraints: ████████████████████ 821   
            free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                 free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           lower: █████████████████⋅⋅⋅ 1571             lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           upper: ████⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 305              upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
         low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0              low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                fixed: ████████████████████ 821   
          infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0               infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            nnzh: (100.00% sparsity)   0               linear: ████████████████████ 821   
                                                    nonlinear: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
                                                         nnzj: ( 99.30% sparsity)   10705 



In [197]:
stats_mpc4 =  mehrotraPCQuadBounds(SM4);
println(stats_mpc4)

┌ Info:   Iter    primal       pdd   rb cond   rc cond      ‖Δx‖         μ  
└ @ Main In[184]:68
┌ Info:  0.0e+00   4.5e+10   1.0e+00   4.1e+10   5.1e+01   0.0e+00   2.2e+09
└ @ Main In[184]:73
┌ Info:  1.0e+00   1.8e+10   1.0e+00   1.4e+10   1.8e+01   1.3e+09   9.0e+08
└ @ Main In[184]:162
┌ Info:  2.0e+00   4.9e+09   1.0e+00   3.6e+09   7.9e+00   5.9e+08   2.5e+08
└ @ Main In[184]:162
┌ Info:  3.0e+00   1.4e+09   1.0e+00   8.7e+08   1.3e+00   2.2e+08   6.2e+07
└ @ Main In[184]:162
┌ Info:  4.0e+00   4.3e+08   1.0e+00   1.5e+08   3.6e-01   1.5e+08   1.1e+07
└ @ Main In[184]:162
┌ Info:  5.0e+00   1.1e+08   1.0e+00   7.1e+06   3.8e-01   1.1e+08   5.2e+05
└ @ Main In[184]:162
┌ Info:  6.0e+00   4.6e+07   1.0e+00   1.1e+06   3.6e-01   6.0e+07   9.0e+04
└ @ Main In[184]:162
┌ Info:  7.0e+00   2.4e+07   1.1e+00   3.9e+05   2.2e-01   3.6e+07   5.1e+04
└ @ Main In[184]:162
┌ Info:  8.0e+00   1.1e+07   1.3e+00   2.2e+05   2.4e-01   2.1e+07   3.3e+04
└ @ Main In[184]:162
┌ Info:  9.0e+00   1.7

┌ Info:  8.6e+01       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[184]:162
┌ Info:  8.7e+01       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[184]:162
┌ Info:  8.8e+01       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[184]:162
┌ Info:  8.9e+01       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[184]:162
┌ Info:  9.0e+01       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[184]:162
┌ Info:  9.1e+01       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[184]:162
┌ Info:  9.2e+01       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[184]:162
┌ Info:  9.3e+01       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[184]:162
┌ Info:  9.4e+01       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[184]:162
┌ Info:  9.5e+01       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[184]:162
┌ Info:  9.6e+01    

┌ Info:  1.7e+02       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[184]:162
┌ Info:  1.8e+02       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[184]:162
┌ Info:  1.8e+02       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[184]:162
┌ Info:  1.8e+02       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[184]:162
┌ Info:  1.8e+02       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[184]:162
┌ Info:  1.8e+02       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[184]:162
┌ Info:  1.8e+02       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[184]:162
┌ Info:  1.8e+02       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[184]:162
┌ Info:  1.8e+02       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[184]:162
┌ Info:  1.8e+02       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[184]:162
┌ Info:  1.8e+02    

┌ Info:  2.6e+02       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[184]:162
┌ Info:  2.6e+02       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[184]:162
┌ Info:  2.6e+02       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[184]:162
┌ Info:  2.6e+02       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[184]:162
┌ Info:  2.6e+02       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[184]:162
┌ Info:  2.6e+02       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[184]:162
┌ Info:  2.6e+02       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[184]:162
┌ Info:  2.7e+02       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[184]:162
┌ Info:  2.7e+02       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[184]:162
┌ Info:  2.7e+02       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[184]:162
┌ Info:  2.7e+02    

Generic Execution stats
  status: maximum iteration
  objective value: NaN
  primal feasibility: NaN
  dual feasibility: NaN
  solution: [NaN  NaN  NaN  NaN ⋯ NaN]
  multipliers: [0.0  NaN  NaN  NaN ⋯ NaN]
  multipliers_L: [NaN  NaN  NaN  NaN ⋯ 0.0]
  multipliers_U: [0.0  0.0  0.0  0.0 ⋯ NaN]
  iterations: 300
  elapsed time: 7.039999961853027


┌ Info:  3.0e+02       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[184]:162
┌ Info:  3.0e+02       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[184]:162


# Tests sur tous les problèmes

In [347]:
df_test = DataFrame(id = Int[],
         name = String[],
         status = Symbol[],
         objective = Float64[],
         time = Float64[],
         iter = Int[],
         primal_feas = Float64[],
         dual_feas = Float64[])



i = 0
i_stop = 4   #number of problems to solve
for file_name in readdir(path_pb)
    if file_name[end-3:end] == ".SIF" && !(file_name in ["80BAU3B.SIF" ; "BORE3D.SIF"; 
                                                        "CAPRI.SIF"; "CZPROB.SIF"; 
                                                        "ETAMACRO.SIF"; "FINNIS.SIF";
                                                        "FORPLAN.SIF"; "GREENBEA.SIF";
                                                        "GREENBEB.SIF"; "MAROS.SIF";
                                                        "NESM.SIF"; "PEROLD.SIF";
                                                         "PILOT-JA.SIF"; "PILOT-WE.SIF";
                                                         "PILOT.SIF"; "PILOT4.SIF";
                                                         "PILOT87.SIF"; "PILOTNOV.SIF";
                                                          "RECIPELP.SIF"; "SHELL.SIF";
                                                         "SIERRA.SIF"; "STAIR.SIF";
                                                         "STANDATA.SIF"; "STANDGUB.SIF";
                                                        "STANDMPS.SIF"; "TUFF.SIF";
                                                        "VTP-BASE.SIF"])   &&# problems with fixed variables
                                        !(file_name in ["25FV47.SIF"; "BNL1.SIF";
                                                        "BRANDY.SIF"; "CRE-A.SIF"]) # singular error
                            
        
        i += 1
        println(file_name)
        pb_i = string(path_pb, "\\", file_name)
        if file_name in ["BLEND.SIF"; "DFL001.SIF"; "FORPLAN.SIF"; "GFRD-PNC.SIF"; "SIERRA.SIF"]
            qpdata_i = readqps(pb_i, mpsformat=:fixed) 
        else
            qpdata_i = readqps(pb_i) 
        end

        SM_i = SlackModel(createQuadraticModel(qpdata_i))

        stats_mpc_i =  mehrotraPCQuadBounds(SM_i, display=false)
        
        # 
        push!(df_test, Dict(:id => i,
                             :name => file_name ,
                             :status => stats_mpc_i.status,
                             :objective => stats_mpc_i.objective,
                             :time => stats_mpc_i.elapsed_time,
                             :iter => stats_mpc_i.iter,
                             :primal_feas => stats_mpc_i.primal_feas,
                             :dual_feas => stats_mpc_i.dual_feas))
    end
    
    if i == i_stop
        break
        break
    end
end

markdown_table(stdout, df_test)

ADLITTLE.SIF
AFIRO.SIF


┌ Info: Using 'ADLITTLE' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using '.Z....' as objective (l. 20)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'ZZZZ0001' as RHS (l. 333)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470
┌ Info: Using 'AFIRO' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'COST' as objective (l. 47)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350


AGG.SIF
AGG2.SIF


┌ Info: Using 'B' as RHS (l. 96)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470
┌ Info: Using 'AGG' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'OBJECTIV' as objective (l. 509)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'AGG' as RHS (l. 1852)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470
┌ Info: Using 'AGG2' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'OBJECTIV' as objective (l. 536)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350


|    id |         name |     status | objective |     time |  iter | primal_fe ⋯
|-------|--------------|------------|-----------|----------|-------|---------- ⋯
|     1 | ADLITTLE.SIF | acceptable |   2.3e+05 |  4.0e-03 |    14 |     1.6e- ⋯
|     2 |    AFIRO.SIF | acceptable |  -4.6e+02 |  1.0e-03 |    11 |     2.3e- ⋯
|     3 |      AGG.SIF | acceptable |  -3.6e+07 |  3.9e-02 |    23 |     5.9e- ⋯
|     4 |     AGG2.SIF | acceptable |  -2.0e+07 |  9.0e-02 |    22 |     1.2e- ⋯


┌ Info: Using 'RHS' as RHS (l. 2878)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470


In [348]:
T = typeof(SM2.meta)
for (name, typ) in zip(fieldnames(T), T.types)
    println("type of the fieldname $name is $typ")
end

type of the fieldname nvar is Int64
type of the fieldname x0 is Array{T,1} where T
type of the fieldname lvar is Array{T,1} where T
type of the fieldname uvar is Array{T,1} where T
type of the fieldname ifix is Array{Int64,1}
type of the fieldname ilow is Array{Int64,1}
type of the fieldname iupp is Array{Int64,1}
type of the fieldname irng is Array{Int64,1}
type of the fieldname ifree is Array{Int64,1}
type of the fieldname iinf is Array{Int64,1}
type of the fieldname nbv is Int64
type of the fieldname niv is Int64
type of the fieldname nlvb is Int64
type of the fieldname nlvo is Int64
type of the fieldname nlvc is Int64
type of the fieldname nlvbi is Int64
type of the fieldname nlvci is Int64
type of the fieldname nlvoi is Int64
type of the fieldname nwv is Int64
type of the fieldname ncon is Int64
type of the fieldname y0 is Array{T,1} where T
type of the fieldname lcon is Array{T,1} where T
type of the fieldname ucon is Array{T,1} where T
type of the fieldname jfix is Array{Int64,1

In [349]:
# problems: AGG.SIF
qpdata5 = readqps(string(path_pb, "\\AGG2.SIF"))
QM5 = createQuadraticModel(qpdata5)
SM5 = SlackModel(QM5);

┌ Info: Using 'AGG2' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'OBJECTIV' as objective (l. 536)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'RHS' as RHS (l. 2878)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470


In [350]:
stats_mpc_5 =  mehrotraPCQuadBounds(SM5, max_iter=150)
println(stats_mpc_5)

┌ Info:   Iter    primal       pdd   rb cond   rc cond      ‖Δx‖         μ  
└ @ Main In[331]:77
┌ Info:  0.0e+00   8.2e+11   1.0e+00   3.1e+10   7.5e+01   0.0e+00   3.9e+09
└ @ Main In[331]:82
┌ Info:  1.0e+00   4.7e+11   1.0e+00   1.2e+10   5.2e+01   1.2e+09   1.8e+09
└ @ Main In[331]:176
┌ Info:  2.0e+00   8.8e+10   1.0e+00   9.2e+08   1.8e+00   9.1e+08   1.8e+08
└ @ Main In[331]:176
┌ Info:  3.0e+00   1.7e+09   1.3e+00   1.8e+07   2.0e-01   1.4e+08   4.1e+06
└ @ Main In[331]:176
┌ Info:  4.0e+00   9.5e+08   1.4e+00   9.6e+06   6.7e-02   1.3e+06   2.6e+06
└ @ Main In[331]:176
┌ Info:  5.0e+00   3.2e+08   1.5e+00   3.4e+06   9.9e-03   1.1e+06   9.1e+05
└ @ Main In[331]:176
┌ Info:  6.0e+00   8.3e+07   2.5e+00   9.5e+05   2.0e-03   4.6e+05   3.5e+05
└ @ Main In[331]:176
┌ Info:  7.0e+00  -2.3e+06   2.2e+01   1.8e+05   2.1e-03   3.0e+05   7.6e+04
└ @ Main In[331]:176
┌ Info:  8.0e+00  -6.2e+06   7.1e+00   1.5e+05   1.7e-03   1.0e+05   6.6e+04
└ @ Main In[331]:176
┌ Info:  9.0e+00  -1.1

Generic Execution stats
  status: solved to within acceptable tolerances
  objective value: -2.0239252353329457e7
  primal feasibility: 1.2096837021380511e-8
  dual feasibility: 5.916584200349462e-8
  solution: [15031.233073265694  0.00023923062989877897  0.0002533786452945386  0.0002637189338962065 ⋯ 547999.9999956683]
  multipliers: [-6.89003112829313e-10  -1.3124969391431051e-8  -2.2428008568508976e-9  -3.6347413421657147e-9 ⋯ 69.78151096967024]
  multipliers_L: [9.141074340835225e-10  0.053999997159137544  0.05099999624509133  0.0489999961548662 ⋯ 0.0]
  multipliers_U: [0.0  0.0  0.0  0.0 ⋯ 2.984999995216971]
  iterations: 

┌ Info:  1.2e+01  -1.5e+07   1.5e+00   4.4e+04   5.4e-04   3.4e+04   3.4e+04
└ @ Main In[331]:176
┌ Info:  1.3e+01  -1.7e+07   1.1e+00   1.2e+04   4.4e-04   4.6e+04   2.6e+04
└ @ Main In[331]:176
┌ Info:  1.4e+01  -1.8e+07   3.0e-01   6.1e+03   2.7e-04   3.6e+04   8.1e+03
└ @ Main In[331]:176
┌ Info:  1.5e+01  -2.0e+07   7.2e-02   1.7e+03   4.5e-04   1.0e+05   2.1e+03
└ @ Main In[331]:176
┌ Info:  1.6e+01  -2.0e+07   1.8e-02   7.2e+01   8.3e-04   2.1e+05   4.9e+02
└ @ Main In[331]:176
┌ Info:  1.7e+01  -2.0e+07   3.9e-03   1.3e+01   3.0e-04   8.2e+04   1.1e+02
└ @ Main In[331]:176
┌ Info:  1.8e+01  -2.0e+07   6.0e-04   1.5e+00   2.2e-04   4.1e+04   1.6e+01
└ @ Main In[331]:176
┌ Info:  1.9e+01  -2.0e+07   6.1e-05   3.8e-02   4.8e-05   9.1e+03   1.7e+00
└ @ Main In[331]:176
┌ Info:  2.0e+01  -2.0e+07   9.1e-06   6.2e-04   7.0e-06   1.1e+03   2.5e-01
└ @ Main In[331]:176
┌ Info:  2.1e+01  -2.0e+07   4.6e-07   1.2e-05   2.0e-06   4.5e+02   1.3e-02
└ @ Main In[331]:176
┌ Info:  2.2e+01  -2

22
  elapsed time: 0.11299991607666016


In [351]:
SM5.meta


  Problem name: qp_pb-slack
   All variables: ████████████████████ 758    All constraints: ████████████████████ 516   
            free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                 free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           lower: ████████⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 302              lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           upper: █████████████⋅⋅⋅⋅⋅⋅⋅ 456              upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
         low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0              low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                fixed: ████████████████████ 516   
          infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0               infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            nnzh: (100.00% sparsity)   0               linear: ████████████████████ 516   
                                                    nonlinear: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
                                                         nnzj: ( 98.79% sparsity)   4740  



In [352]:
SM2.meta.iupp

19-element Array{Int64,1}:
 33
 34
 35
 36
 37
 38
 39
 40
 41
 42
 43
 44
 45
 46
 47
 48
 49
 50
 51

In [18]:
problems = []
i_max = 5
i = 1
for file_name in readdir(path_pb)
    if file_name[end-3:end] == ".SIF" && !(file_name in ["80BAU3B.SIF" ; "BORE3D.SIF"; 
                                                        "CAPRI.SIF"; "CZPROB.SIF"; 
                                                        "ETAMACRO.SIF"; "FINNIS.SIF";
                                                        "FORPLAN.SIF"; "GREENBEA.SIF";
                                                        "GREENBEB.SIF"; "MAROS.SIF";
                                                        "NESM.SIF"; "PEROLD.SIF";
                                                         "PILOT-JA.SIF"; "PILOT-WE.SIF";
                                                         "PILOT.SIF"; "PILOT4.SIF";
                                                         "PILOT87.SIF"; "PILOTNOV.SIF";
                                                          "RECIPELP.SIF"; "SHELL.SIF";
                                                         "SIERRA.SIF"; "STAIR.SIF";
                                                         "STANDATA.SIF"; "STANDGUB.SIF";
                                                        "STANDMPS.SIF"; "TUFF.SIF";
                                                        "VTP-BASE.SIF"])   # problems with fixed variables


        println(file_name)
        pb_i = string(path_pb, "\\", file_name)
        if file_name in ["BLEND.SIF"; "DFL001.SIF"; "FORPLAN.SIF"; "GFRD-PNC.SIF"; "SIERRA.SIF"]
            qpdata_i = readqps(pb_i, mpsformat=:fixed) 
        else
            qpdata_i = readqps(pb_i) 
        end
        push!(problems, SlackModel(createQuadraticModel(qpdata_i, name=file_name[1:end-4])))
        
        if i==i_max
            break
        end
        i += 1
    end
end


25FV47.SIF


┌ Info: Using '25FV47' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751


ADLITTLE.SIF
AFIRO.SIF
AGG.SIF
AGG2.SIF


┌ Info: Using 'R0000' as objective (l. 21)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using '.00001' as RHS (l. 6793)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470
┌ Info: Using 'ADLITTLE' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using '.Z....' as objective (l. 20)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'ZZZZ0001' as RHS (l. 333)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470
┌ Info: Using 'AFIRO' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'COST' as objective (l. 47)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'B' as RHS (l. 96)
└ @ QPSReader C:\Users\Geoffroy Leconte\.ju

In [19]:
problems[2].meta.name

"ADLITTLE-slack"

In [20]:
problems_stats = solve_problems(mehrotraPCQuadBounds, problems, 
                                colstats=[:name, :status, :elapsed_time, :objective, :dual_feas, :primal_feas])

┌ Info:            Name           status      Time      f(x)      Dual    Primal  
└ @ SolverTools C:\Users\Geoffroy Leconte\.julia\packages\SolverTools\whyGk\src\bmark\run_solver.jl:67
┌ Info:    25FV47-slack       acceptable   8.2e-01   5.5e+03   4.1e-08   9.1e-11
└ @ SolverTools C:\Users\Geoffroy Leconte\.julia\packages\SolverTools\whyGk\src\bmark\run_solver.jl:83
┌ Info:  ADLITTLE-slack         max_iter   2.1e-01       NaN       NaN       NaN
└ @ SolverTools C:\Users\Geoffroy Leconte\.julia\packages\SolverTools\whyGk\src\bmark\run_solver.jl:83
┌ Info:     AFIRO-slack       acceptable   2.0e-03  -4.6e+02   1.6e-09   4.4e-11
└ @ SolverTools C:\Users\Geoffroy Leconte\.julia\packages\SolverTools\whyGk\src\bmark\run_solver.jl:83
┌ Info:       AGG-slack       acceptable   2.4e-01  -3.6e+07   1.0e-08   2.0e-07
└ @ SolverTools C:\Users\Geoffroy Leconte\.julia\packages\SolverTools\whyGk\src\bmark\run_solver.jl:83
┌ Info:      AGG2-slack       acceptable   9.2e-01  -2.0e+07   3.4e-07   1.4e-

,id,name,nvar,ncon,nequ,status,objective,elapsed_time
,Int64,String,Int64,Int64,Int64,Symbol,Float64,Float64
1,1,25FV47-slack,1876,821,0,acceptable,5501.85,0.817
2,2,ADLITTLE-slack,138,56,0,max_iter,NaN,0.213
3,3,AFIRO-slack,51,27,0,acceptable,-464.753,0.00200009
4,4,AGG-slack,615,488,0,acceptable,-3.59918e7,0.238
5,5,AGG2-slack,758,516,0,acceptable,-2.02393e7,0.919


In [21]:
markdown_table(stdout, problems_stats)

|    id |           name |  nvar |  ncon |  nequ |     status | objective | el ⋯
|-------|----------------|-------|-------|-------|------------|-----------|--- ⋯
|     1 |   25FV47-slack |  1876 |   821 |     0 | acceptable |   5.5e+03 |    ⋯
|     2 | ADLITTLE-slack |   138 |    56 |     0 |   max_iter |       NaN |    ⋯
|     3 |    AFIRO-slack |    51 |    27 |     0 | acceptable |  -4.6e+02 |    ⋯
|     4 |      AGG-slack |   615 |   488 |     0 | acceptable |  -3.6e+07 |    ⋯
|     5 |     AGG2-slack |   758 |   516 |     0 | acceptable |  -2.0e+07 |    ⋯


In [121]:
using JLD2

In [211]:
jldopen("mehrotra.jld2", "w") do file
    file["test"] = problems_stats
end

,id,name,nvar,ncon,nequ,status,objective,elapsed_time
,Int64,String,Int64,Int64,Int64,Symbol,Float64,Float64
1,1,25FV47-slack,1876,821,0,max_iter,NaN,7.158
2,2,ADLITTLE-slack,138,56,0,acceptable,225495.0,0.0110002
3,3,AFIRO-slack,51,27,0,acceptable,-464.753,0.00399995
4,4,AGG-slack,615,488,0,acceptable,-3.59918e7,0.063
5,5,AGG2-slack,758,516,0,acceptable,-2.02393e7,0.234


In [119]:
res_path = "C:\\Users\\Geoffroy Leconte\\Documents\\cours\\TFE\\code\\results"

"C:\\Users\\Geoffroy Leconte\\Documents\\cours\\TFE\\code\\results"

In [123]:
file = jldopen(string(res_path, "\\mehrotra_ldlt1.jld2"), "r")

JLDFile C:\Users\Geoffroy Leconte\Documents\cours\TFE\code\results\mehrotra_ldlt1.jld2 (read-only)
 └─🔢 stats

In [148]:
withenv("COLUMNS" => 600, "LINES" => 200) do
    display(file["stats"])
end

,id,name,nvar,ncon,nequ,status,objective,elapsed_time,iter,dual_feas,primal_feas,neval_obj,neval_grad,neval_cons,neval_jcon,neval_jgrad,neval_jac,neval_jprod,neval_jtprod,neval_hess,neval_hprod,neval_jhprod,neval_residual,neval_jac_residual,neval_jprod_residual,neval_jtprod_residual,neval_hess_residual,neval_jhess_residual,neval_hprod_residual,extrainfo
,Int64,String,Int64,Int64,Int64,Symbol,Float64,Float64,Int64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,String
1,1,25FV47-slack,1876,821,0,max_iter,NaN,14.4245,1000,NaN,NaN,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,
2,2,ADLITTLE-slack,138,56,0,acceptable,225495.0,0.00607681,14,1.83289e-8,1.63805e-7,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,
3,3,AFIRO-slack,51,27,0,acceptable,-464.753,0.00186396,11,7.20535e-14,2.27374e-13,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,
4,4,AGG-slack,615,488,0,acceptable,-3.59918e7,0.0545862,23,2.36025e-10,5.93841e-9,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,
5,5,AGG2-slack,758,516,0,acceptable,-2.02393e7,0.11677,22,5.91658e-8,1.20968e-8,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,
6,6,AGG3-slack,758,516,0,exception,Inf,Inf,0,Inf,Inf,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,ZeroPivotException(1)
7,7,BANDM,472,305,0,max_iter,4.98602e10,2.13335,1000,0.001,0.0686481,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,
8,8,BEACONFD-slack,295,173,0,acceptable,33592.5,0.0201521,10,7.80034e-11,3.55019e-8,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,
9,9,BLEND-slack,114,74,0,acceptable,-30.8121,0.00719619,14,3.42152e-13,1.87046e-12,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,
